<a href="https://colab.research.google.com/github/Amma-Anjali/Generative_AI_2025/blob/main/6_3_gen_ai_2303A52385.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load dataset
dataset_url = "https://drive.google.com/uc?id=1AcdENlVm5dccNyo_vgdMbneX8YVvH5R3"
df = pd.read_csv(dataset_url)

# Assuming the last column is the target variable
y = df.iloc[:, -1].values
X = df.iloc[:, :-1]  # Keep X as a DataFrame

# Convert the target variable to numerical representation if it contains strings
if df.iloc[:, -1].dtype == 'object':  # Check if target variable is categorical
    le = LabelEncoder()
    y = le.fit_transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical features
categorical_features = df.select_dtypes(include=['object']).columns[:-1]  # Exclude the target variable
numerical_features = df.select_dtypes(exclude=['object']).columns

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features),
    ]
)

# Transform dataset
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Define ANN model
model = Sequential([
    Dense(18, activation='swish', input_shape=(X_train.shape[1],)),
    Dense(26, activation='swish'),
    Dense(20, activation='swish'),
    Dense(15, activation='swish'),
    Dense(1, activation='linear')
])

# Compile model
model.compile(optimizer=RMSprop(), loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# Save model
model.save("housing_price_model.h5")

# Predict and calculate error metrics
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Training MAE: {train_mae}")
print(f"Testing MAE: {test_mae}")
print(f"Training RMSE: {train_rmse}")
print(f"Testing RMSE: {test_rmse}")

# Load and compile the model before inference
loaded_model = keras.models.load_model("housing_price_model.h5")
loaded_model.compile(optimizer=RMSprop(), loss='mean_squared_error')

# Function for making predictions
def predict_price(input_data):
    input_df = pd.DataFrame([input_data], columns=X.columns)  # Keep original column names
    input_transformed = preprocessor.transform(input_df)  # Transform input data
    return loaded_model.predict(input_transformed)[0, 0]

# Example prediction
sample_input = X.iloc[0].to_dict()  # Get a sample from original X as a dictionary
predicted_price = predict_price(sample_input)

print(f"Predicted Price: {predicted_price}")

